# Transferencia de conocimiento

En esta segunda práctica de transferencia de conocimiento vamos a practicar con distintas arquitecturas. Para ello vamos a emplear la base de datos de clasificación de imágenes en pnumonía vs normal: https://drive.google.com/file/d/1wgODntXVXLp5bS0IyP9HVpGOrykNpS3h/view

Para la práctica debes de realizar los siguientes ejercicios:


In [5]:
from pathlib import Path

import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.ticker import MultipleLocator
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

IMG_SIZE = (256, 256, 3)

BATCH_SIZE = 32
RANDOM_STATE = 123

# Ejercicio 1

Descarga la base de datos y prepara los datasets de entrenamiento y validación con un tamaño de imagen de 256x256 y un tamaño de lote de 32.

In [6]:
def get_images():
    path_dataset = Path('./chest_xray')
    path_train = path_dataset / 'entrenamiento'
    path_val = path_dataset / 'validacion'

    train_dataset = image_dataset_from_directory(
        path_train,
        shuffle=True,
        seed=RANDOM_STATE,
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE[0], IMG_SIZE[1])
    )

    validation_dataset = image_dataset_from_directory(
        path_val,
        shuffle=True,
        seed=RANDOM_STATE,
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE[0], IMG_SIZE[1])
    )

    return train_dataset, validation_dataset


train_dataset, validation_dataset = get_images()

Found 500 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


# Ejercicio 2

Prepara una función que va a recibir el nombre de la arquitectura que vamos a emplear para realizar transferencia de conocimiento. Vamos a probar con las siguientes arquitecturas: 'densenet', 'efficientnet_v2', 'inception_v3', 'mobilenet_v3', 'resnet_v2' y 'xception'.

La función recibirá el nombre de la arquitectura como palabra clave. En función de esta, la función devolverá el modelo pre-entrenado (con los pesos de ImageNet y sin la última capa de clasificación) y la capa de pre-proceso correspondientes.

In [7]:
def get_model(model):
    match model:
        case 'densenet':
            model_constructor = tf.keras.applications.densenet.DenseNet121
            preprocess = tf.keras.application.densenet.preprocess_input
        case 'efficientnet_v2':
            model_constructor = tf.keras.applications.efficientnet_v2.EfficientNetV2S
            preprocess = tf.keras.application.efficientnet_v2.preprocess_input
        case 'inception_v3':
            model_constructor = tf.keras.applications.inceptiov_v3.InceptionV3
            preprocess = tf.keras.application.inceptiov_v3.preprocess_input
        case 'mobilnet_v3':
            model_constructor = tf.keras.applications.MobileNetV3Small
            preprocess = tf.keras.application.mobilenet_v3.preprocess_input
        case 'resnet_v2':
            model_constructor = tf.keras.applications.resnet_v2.ResNet101V2
            preprocess = tf.keras.application.resnet_v2.preprocess_input
        case 'xception':
            model_constructor = tf.keras.applications.xception.Xception
            preprocess = tf.keras.application.xception.Xception
        case _:
            print('Model does not exit')
            exit(1)

    model_base = model_constructor(include_top=False, weights='imagenet', input_shape=IMG_SIZE)

    return model_base, preprocess

# Ejercicio 3

Construye una función que, reciba: el tamaño de la imagen de entrada, la capa de preproceso y el modelo pre-entrenado y, tras añadirle la capa densa correspondiente, devuelva el modelo que vamos a emplear para resolver el problema de clasificación.
El modelo pre-entrenado tendrá todas las capas congeladas.



In [8]:
def add_first_layer(image_size, preprocess, model):
    model.trainable = False

    input_layer = layers.Input(image_size)
    preprocess_layer = preprocess(input_layer)
    base_layer = model(preprocess_layer, training=False)
    output_layer = layers.Dense(1, activation="sigmoid")(base_layer)

    model_built = Model(inputs=[input_layer], outputs=[output_layer])

    return model_built

# Ejercicio 4

Construye una función que, dado un modelo, lo entrene durante 10 épocas y, tras este entrenamiento, descongele todo el modelo y congele solo la primera mitad de las capas y vuelva a entrenar durante 5 épocas más con una tasa de aprendizaje de 10^-5. Esta función deberá devolver la historia del entrenamiento.

In [8]:
def train_model(model, train_data, validation_data):
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='./Practice04.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=True
    )

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history_extraction = model.fit(train_data, epochs=10, validation_data=validation_dataset,
                                   callbacks=[model_checkpoint_callback], verbose=True)

    model.trainable = True
    model.compile(optimizer='adam', learning_rate=1e-5, loss='binary_crossentropy', metrics=['accuracy'])

    history_fine_tuning = model.fit(train_data, epochs=5, validation_data=validation_dataset,
                                    callbacks=[model_checkpoint_callback], verbose=True)

    return history_fine_tuning


# Ejercicio 5

Ejecuta la función anterior para cada uno de los modelos definidos en el ejercicio 2. Para cada uno de ellos muestra la curvas de entrenamiento y validación y, tras todos los entrenamiento, muestre en una sola gráfica todas las curvas de validación.

¿Con qué modelo te quedarías?